# Example: Rotation in BFV

Both the BFV and BGV schemes (with BatchEncoder) as well as the CKKS scheme support
native vectorized computations on encrypted numbers. In addition to computing slot-wise,
it is possible to rotate the encrypted vectors cyclically.

Simply changing `scheme_type.bfv` to `scheme_type.bgv` will make this example work for
the BGV scheme.

In [1]:
import seal
import example_helper

parms = seal.EncryptionParameters(seal.scheme_type.bfv)
poly_modulus_degree = 8192
parms.poly_modulus_degree = poly_modulus_degree
parms.coeff_modulus = seal.CoeffModulus.BFVDefault(poly_modulus_degree)
parms.plain_modulus = seal.PlainModulus.Batching(poly_modulus_degree, 20)

context = seal.SEALContext(parms)
example_helper.print_parameters(context)

/
| Encryption parameters
| scheme: bfv
| poly_modulus_degree: 8192
| coeff_modulus size: 218(43 + 43 + 44 + 44 + 44) bits
| plain_modulus: 1032193
\


In [2]:
keygen = seal.KeyGenerator(context)
secret_key = keygen.secret_key
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
encryptor = seal.Encryptor(context, public_key)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, secret_key)

batch_encoder = seal.BatchEncoder(context)
slot_count = batch_encoder.slot_count
row_size = slot_count // 2
print(f"Plaintext matrix row size: {row_size}")

Plaintext matrix row size: 4096


In [3]:
pod_matrix = [0 for _ in range(slot_count)]
pod_matrix[0] = 0
pod_matrix[1] = 1
pod_matrix[2] = 2
pod_matrix[3] = 3
pod_matrix[row_size] = 4
pod_matrix[row_size+1] = 5
pod_matrix[row_size+2] = 6
pod_matrix[row_size+3] = 7

print("Input plaintext matrix:")
example_helper.print_matrix(pod_matrix, row_size)

Input plaintext matrix:

    [ 0.000, 1.000, 2.000, 3.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]
    [ 4.000, 5.000, 6.000, 7.000, 0.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]



First we use BatchEncoder to encode the matrix into a plaintext. We encrypt
the plaintext as usual.

In [4]:
plain_matrix = batch_encoder.encode(pod_matrix)
encrypted_matrix = encryptor.encrypt(plain_matrix)
print(f"noise budget in encrypted_matrix: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")

noise budget in encrypted_matrix: 146 bits


Rotations require yet another type of special key called `Galois keys`. These
are easily obtained from the KeyGenerator.

In [5]:
galois_keys = keygen.create_galois_keys()

Now rotate both matrix rows 3 steps to the left, decrypt, decode, and print.

In [6]:
print("Rotate rows 3 steps left.")
evaluator.rotate_rows_inplace(encrypted_matrix, 3, galois_keys)
print(f"    + Noise budget after rotation: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")
print(f"    + Decrypt and decode")
plain_result = decryptor.decrypt(encrypted_matrix)
pod_matrix = batch_encoder.decode(plain_result)
example_helper.print_matrix(pod_matrix, row_size)

Rotate rows 3 steps left.
    + Noise budget after rotation: 142 bits
    + Decrypt and decode

    [ 3.000, 0.000, 0.000, 0.000, 0.000, ..., 0.000, 0.000, 0.000, 1.000, 2.000 ]
    [ 7.000, 0.000, 0.000, 0.000, 0.000, ..., 0.000, 0.000, 4.000, 5.000, 6.000 ]



We can also rotate the columns, i.e., swap the rows.

In [7]:
print("Rotate columns.")
evaluator.rotate_columns_inplace(encrypted_matrix, galois_keys)
print(f"    + Noise budget after rotation: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")
print(f"    + Decrypt and decode")
plain_result = decryptor.decrypt(encrypted_matrix)
pod_matrix = batch_encoder.decode(plain_result)
example_helper.print_matrix(pod_matrix, row_size)

Rotate columns.
    + Noise budget after rotation: 142 bits
    + Decrypt and decode

    [ 7.000, 0.000, 0.000, 0.000, 0.000, ..., 0.000, 0.000, 4.000, 5.000, 6.000 ]
    [ 3.000, 0.000, 0.000, 0.000, 0.000, ..., 0.000, 0.000, 0.000, 1.000, 2.000 ]



Finally, we rotate the rows 4 steps to the right, decrypt, decode, and print.

In [8]:
print("Rotate rows 4 steps right.")
evaluator.rotate_rows_inplace(encrypted_matrix, -4, galois_keys)
print(f"    + Noise budget after rotation: {decryptor.invariant_noise_budget(encrypted_matrix)} bits")
print(f"    + Decrypt and decode")
plain_result = decryptor.decrypt(encrypted_matrix)
pod_matrix = batch_encoder.decode(plain_result)
example_helper.print_matrix(pod_matrix, row_size)

Rotate rows 4 steps right.
    + Noise budget after rotation: 142 bits
    + Decrypt and decode

    [ 0.000, 4.000, 5.000, 6.000, 7.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]
    [ 0.000, 0.000, 1.000, 2.000, 3.000, ..., 0.000, 0.000, 0.000, 0.000, 0.000 ]



Note that rotations do not consume any noise budget. However, this is only
the case when the special prime is at least as large as the other primes. The
same holds for relinearization. Microsoft SEAL does not require that the
special prime is of any particular size, so ensuring this is the case is left
for the user to do.